In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import calendar
from matplotlib import pyplot as plt
import scipy.stats as stats
import glob

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from mpl_toolkits.mplot3d import Axes3D  #3Dplot
import seaborn as sns
import japanize_matplotlib


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from matplotlib.ticker import ScalarFormatter
import matplotlib.ticker as ptick ##これが必要！

In [2]:
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})



In [ ]:
# 分散分析


In [8]:


cols = ['DateTime', 'gap_price[intra-spot](円/kWh)','低下量合計', '停止・低下ユニット数合計','intra_volume（MWh/h）','spot_volume(kWh)', '太陽光絶対値誤差合算(実績-想定)(kWh)',
       '風力絶対値誤差合算(実績-想定)(kWh)']

df_select = df_king[cols]

re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)','出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
       '風力誤差(実績-想定)合算値 (kWh)'
    #    , 'エリア毎の広域予備率の平均値(%)'
       ]
df_select.columns = re_col
df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

percent =50
# これは各年
for year in range(2018,2023):
# year=2022
    start_point = dt.datetime(year,1,1,0,0)
    end_point = dt.datetime(year,12,31,23,30)

    # まず、データがある分すべての相関をとる。
    target =re_col[2:]
    for name in target:
        df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
        df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


        df['judge'] =0

        df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
        judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

        df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

        value0 = df[df['judge']==0][name].values
        value1 = df[df['judge']==1][name].values
        times =df['DateTime'].dt.date.values

        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, alternative='two-sided',equal_var=False)
        print("t_value:", t_value)
        print("p_value:", p_value)
    
        
        if p_value < 0.05/6/5:
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい")
        else:
            # print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")
            pass


C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_22048\1907662825.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2



2018 出力低下量合計 (kW)
t_value: 18.43795948648608
p_value: 5.004385517544802e-75
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2018 停止・低下ユニット数合計 (基)
t_value: 17.30831226953067
p_value: 1.9900462654324457e-66
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2018 時間前市場取引量 (kWh)
t_value: -29.13282404307496
p_value: 1.7412194954218761e-180
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2018 スポット市場取引量 (kWh)
t_value: -10.539034654250036
p_value: 7.130550272000707e-26
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2018 太陽光誤差(実績-想定)合算 (kWh)
t_value: -0.13256649466207215
p_value: 0.8945381466381971

2018 風力誤差(実績-想定)合算値 (kWh)
t_value: -4.8770461746517695
p_value: 1.08902639931136e-06
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2019 出力低下量合計 (kW)
t_value: -2.2291705612575683
p_value: 0.02581518960049367

2019 停止・低下ユニット数合計 (基)
t_value: 1.0337057405848813
p_value: 0.30128801943190064

2019 時間前市場取引量 (kWh)
t_value: -20.15051351922978
p_value: 3.799532922731972e-89
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2019 スポット市場取引量 (kWh)
t

In [48]:
print('等分散を仮定')
#片側t検定　ある方がない方より大きい 
#１年毎正式版
cols = ['DateTime', 'gap_price[intra-spot](円/kWh)','低下量合計', '停止・低下ユニット数合計','intra_volume（MWh/h）','spot_volume(kWh)', '太陽光絶対値誤差合算(実績-想定)(kWh)',
       '風力絶対値誤差合算(実績-想定)(kWh)']

df_select = df_king[cols]

re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)','出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
       '風力誤差(実績-想定)合算値 (kWh)'
    #    , 'エリア毎の広域予備率の平均値(%)'
       ]
df_select.columns = re_col
df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

percent =50
# これは各年

# for year in range(2018,2023):

for year in range(2018,2023):
# year=2022
    start_point = dt.datetime(year,1,1,0,0)
    end_point = dt.datetime(year,12,31,23,30)

    # まず、データがある分すべての相関をとる。
    target =re_col[2:]
    for name in target:
        df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
        df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


        df['judge'] =0

        df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
        judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

        df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

        value0 = df[df['judge']==0][name].values
        value1 = df[df['judge']==1][name].values
        times =df['DateTime'].dt.date.values

        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, alternative='less',equal_var=True)
        print("t_value:", t_value)
        print("p_value:", p_value)
    
        
        if p_value < 0.05:
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい")
        else:
            print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")

print('')


print('')

print('')

print('')

print('')

print('')

print('等分散を仮定しない（welchのｔ）')


for year in range(2018,2023):
# year=2022
    start_point = dt.datetime(year,1,1,0,0)
    end_point = dt.datetime(year,12,31,23,30)

    # まず、データがある分すべての相関をとる。
    target =re_col[2:]
    for name in target:
        df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
        df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


        df['judge'] =0

        df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
        judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

        df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

        value0 = df[df['judge']==0][name].values
        value1 = df[df['judge']==1][name].values
        times =df['DateTime'].dt.date.values

        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, alternative='less',equal_var=False)
        print("t_value:", t_value)
        print("p_value:", p_value)
    
        
        if p_value < 0.05:
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい")
        else:
            print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")


等分散を仮定

2018 出力低下量合計 (kW)
t_value: 18.437687589920372
p_value: 1.0
1.000 のため、帰無仮説が採択されました。あるとないに差はありません

2018 停止・低下ユニット数合計 (基)
t_value: 17.308419254339952
p_value: 1.0
1.000 のため、帰無仮説が採択されました。あるとないに差はありません

2018 時間前市場取引量 (kWh)
t_value: -29.139075810306448
p_value: 1.3580744750332593e-181
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2018 スポット市場取引量 (kWh)
t_value: -10.53907229197732
p_value: 3.5638166121531384e-26
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2018 太陽光誤差(実績-想定)合算 (kWh)
t_value: -0.1325662380131005
p_value: 0.44726917479696376
0.447 のため、帰無仮説が採択されました。あるとないに差はありません

2018 風力誤差(実績-想定)合算値 (kWh)
t_value: -4.877362065542579
p_value: 5.435073141678228e-07
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2019 出力低下量合計 (kW)
t_value: -2.2291705612575683
p_value: 0.012907594707504725
p = 0.013 のため、帰無仮説が棄却されました。ある より ない　は大きい

2019 停止・低下ユニット数合計 (基)
t_value: 1.0337057405848815
p_value: 0.8493559907776304
0.849 のため、帰無仮説が採択されました。あるとないに差はありません

2019 時間前市場取引量 (kWh)
t_value: -20.15051351922978
p_value: 1.3667

C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_22392\1703579776.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2


t_value: -18.88688844422721
p_value: 4.4271243411634513e-79
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2021 停止・低下ユニット数合計 (基)
t_value: -16.062451709485583
p_value: 6.032409596064562e-58
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2021 時間前市場取引量 (kWh)
t_value: -27.620187844742514
p_value: 1.0385718965899464e-164
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2021 スポット市場取引量 (kWh)
t_value: 0.03263628057579847
p_value: 0.5130174954101057
0.513 のため、帰無仮説が採択されました。あるとないに差はありません

2021 太陽光誤差(実績-想定)合算 (kWh)
t_value: 4.072467151026489
p_value: 0.9999766389031695
1.000 のため、帰無仮説が採択されました。あるとないに差はありません

2021 風力誤差(実績-想定)合算値 (kWh)
t_value: -9.511089582848559
p_value: 1.0624911382061362e-21
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2022 出力低下量合計 (kW)
t_value: 10.901851352601676
p_value: 1.0
1.000 のため、帰無仮説が採択されました。あるとないに差はありません

2022 停止・低下ユニット数合計 (基)
t_value: 9.453305523554961
p_value: 1.0
1.000 のため、帰無仮説が採択されました。あるとないに差はありません

2022 時間前市場取引量 (kWh)
t_value: -23.03428125083708
p_value: 5.507354610308577e-116
p = 0.000 

In [50]:
print('等分散を仮定')
#片側t検定　ある方がない方より大きい 
#１年毎正式版
cols = ['DateTime', 'gap_price[intra-spot](円/kWh)','低下量合計', '停止・低下ユニット数合計','intra_volume（MWh/h）','spot_volume(kWh)', '太陽光絶対値誤差合算(実績-想定)(kWh)',
       '風力絶対値誤差合算(実績-想定)(kWh)']

df_select = df_king[cols]

re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)','出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
       '風力誤差(実績-想定)合算値 (kWh)'
    #    , 'エリア毎の広域予備率の平均値(%)'
       ]
df_select.columns = re_col
df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

percent =50
# これは各年

# for year in range(2018,2023):

for year in range(2018,2023):
# year=2022
    start_point = dt.datetime(year,1,1,0,0)
    end_point = dt.datetime(year,12,31,23,30)

    # まず、データがある分すべての相関をとる。
    target =re_col[2:]
    for name in target:
        df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
        df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


        df['judge'] =0

        df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
        judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

        df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

        value0 = df[df['judge']==0][name].values
        value1 = df[df['judge']==1][name].values
        times =df['DateTime'].dt.date.values

        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, alternative='less',equal_var=True)
        print("t_value:", t_value)
        print("p_value:", p_value)
    
        
        if p_value < 0.05:
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい")
        else:
            # print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")
            pass

print('')


print('')

print('')

print('')

print('')

print('')

print('等分散を仮定しない（welchのｔ）')


for year in range(2018,2023):
# year=2022
    start_point = dt.datetime(year,1,1,0,0)
    end_point = dt.datetime(year,12,31,23,30)

    # まず、データがある分すべての相関をとる。
    target =re_col[2:]
    for name in target:
        df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
        df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


        df['judge'] =0

        df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
        judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

        df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

        value0 = df[df['judge']==0][name].values
        value1 = df[df['judge']==1][name].values
        times =df['DateTime'].dt.date.values

        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, alternative='less',equal_var=False)
        print("t_value:", t_value)
        print("p_value:", p_value)
    
        
        if p_value < 0.05:
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい")
        else:
            # print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")
            pass


等分散を仮定

2018 出力低下量合計 (kW)
t_value: 18.437687589920372
p_value: 1.0

2018 停止・低下ユニット数合計 (基)
t_value: 17.308419254339952
p_value: 1.0

2018 時間前市場取引量 (kWh)
t_value: -29.139075810306448
p_value: 1.3580744750332593e-181
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2018 スポット市場取引量 (kWh)
t_value: -10.53907229197732
p_value: 3.5638166121531384e-26
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2018 太陽光誤差(実績-想定)合算 (kWh)
t_value: -0.1325662380131005
p_value: 0.44726917479696376

2018 風力誤差(実績-想定)合算値 (kWh)
t_value: -4.877362065542579
p_value: 5.435073141678228e-07
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2019 出力低下量合計 (kW)
t_value: -2.2291705612575683
p_value: 0.012907594707504725
p = 0.013 のため、帰無仮説が棄却されました。ある より ない　は大きい

2019 停止・低下ユニット数合計 (基)
t_value: 1.0337057405848815
p_value: 0.8493559907776304

2019 時間前市場取引量 (kWh)
t_value: -20.15051351922978
p_value: 1.3667648010742497e-89
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2019 スポット市場取引量 (kWh)
t_value: -1.4629910891368731
p_value: 0.07174387897480716

2019 太陽光誤

C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_22392\10881816.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2



2021 スポット市場取引量 (kWh)
t_value: 0.03263628057579847
p_value: 0.5130174954101057

2021 太陽光誤差(実績-想定)合算 (kWh)
t_value: 4.072467151026489
p_value: 0.9999766389031695

2021 風力誤差(実績-想定)合算値 (kWh)
t_value: -9.511089582848559
p_value: 1.0624911382061362e-21
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2022 出力低下量合計 (kW)
t_value: 10.901851352601676
p_value: 1.0

2022 停止・低下ユニット数合計 (基)
t_value: 9.453305523554961
p_value: 1.0

2022 時間前市場取引量 (kWh)
t_value: -23.03428125083708
p_value: 5.507354610308577e-116
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2022 スポット市場取引量 (kWh)
t_value: -20.7573973298456
p_value: 7.210502436552569e-95
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2022 太陽光誤差(実績-想定)合算 (kWh)
t_value: -18.64200519953641
p_value: 7.151016510522198e-77
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい

2022 風力誤差(実績-想定)合算値 (kWh)
t_value: 1.2091872693462224
p_value: 0.8866936366068005






等分散を仮定しない（welchのｔ）

2018 出力低下量合計 (kW)
t_value: 18.43795948648608
p_value: 1.0

2018 停止・低下ユニット数合計 (基)
t_value: 17.30831226953067
p

In [4]:
# 両側検定
#１年毎正式版
cols = ['DateTime', 'gap_price[intra-spot](円/kWh)','低下量合計', '停止・低下ユニット数合計','intra_volume（MWh/h）','spot_volume(kWh)', '太陽光絶対値誤差合算(実績-想定)(kWh)',
       '風力絶対値誤差合算(実績-想定)(kWh)']

df_select = df_king[cols]

re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)','出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
       '風力誤差(実績-想定)合算値 (kWh)'
    #    , 'エリア毎の広域予備率の平均値(%)'
       ]
df_select.columns = re_col
df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

percent =50
# これは各年

# for year in range(2018,2023):
# 分散が等しくないとき
for year in range(2018,2023):
# year=2022
    start_point = dt.datetime(year,1,1,0,0)
    end_point = dt.datetime(year,12,31,23,30)

    # まず、データがある分すべての相関をとる。
    target =re_col[2:]
    for name in target:
        df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
        df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


        df['judge'] =0

        df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
        judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

        df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

        value0 = df[df['judge']==0][name].values
        value1 = df[df['judge']==1][name].values
        times =df['DateTime'].dt.date.values

        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, equal_var=True,alternative='two-sided')
        print("t_value:", t_value)
        print("p_value:", p_value)
    
        
        if p_value < 0.025:
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。AとBに差があります")
        else:
            print(f"{p_value:.3f} のため、帰無仮説が採択されました。AとBに差はありません")

C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_24172\2922962599.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2



2018 出力低下量合計 (kW)
t_value: 18.437687589920372
p_value: 5.02284394041912e-75
p = 0.000 のため、帰無仮説が棄却されました。AとBに差があります

2018 停止・低下ユニット数合計 (基)
t_value: 17.308419254339952
p_value: 1.9859831395678503e-66
p = 0.000 のため、帰無仮説が棄却されました。AとBに差があります

2018 時間前市場取引量 (kWh)
t_value: -29.139075810306448
p_value: 2.7161489500665185e-181
p = 0.000 のため、帰無仮説が棄却されました。AとBに差があります

2018 スポット市場取引量 (kWh)
t_value: -10.53907229197732
p_value: 7.127633224306277e-26
p = 0.000 のため、帰無仮説が棄却されました。AとBに差があります

2018 太陽光誤差(実績-想定)合算 (kWh)
t_value: -0.1325662380131005
p_value: 0.8945383495939275
0.895 のため、帰無仮説が採択されました。AとBに差はありません

2018 風力誤差(実績-想定)合算値 (kWh)
t_value: -4.877362065542579
p_value: 1.0870146283356455e-06
p = 0.000 のため、帰無仮説が棄却されました。AとBに差があります

2019 出力低下量合計 (kW)
t_value: -2.2291705612575683
p_value: 0.02581518941500945
0.026 のため、帰無仮説が採択されました。AとBに差はありません

2019 停止・低下ユニット数合計 (基)
t_value: 1.0337057405848815
p_value: 0.30128801844473935
0.301 のため、帰無仮説が採択されました。AとBに差はありません

2019 時間前市場取引量 (kWh)
t_value: -20.15051351922978
p_va

In [51]:
print('等分散を仮定')

#片側t検定　ある方がない方より大きい 
#１年毎正式版
cols = ['DateTime', 'gap_price[intra-spot](円/kWh)','低下量合計', '停止・低下ユニット数合計','intra_volume（MWh/h）','spot_volume(kWh)', '太陽光絶対値誤差合算(実績-想定)(kWh)',
       '風力絶対値誤差合算(実績-想定)(kWh)']

df_select = df_king[cols]

re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)','出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
       '風力誤差(実績-想定)合算値 (kWh)'
    #    , 'エリア毎の広域予備率の平均値(%)'
       ]
df_select.columns = re_col
df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

percent =50
# これは各年

# for year in range(2018,2023):


for year in range(2018,2023):
# year=2022
    start_point = dt.datetime(year,1,1,0,0)
    end_point = dt.datetime(year,12,31,23,30)

    # まず、データがある分すべての相関をとる。
    target =re_col[2:]
    for name in target:
        df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
        df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


        df['judge'] =0

        df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
        judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

        df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

        value0 = df[df['judge']==0][name].values
        value1 = df[df['judge']==1][name].values
        times =df['DateTime'].dt.date.values

        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, alternative='greater',equal_var=True)
        print("t_value:", t_value)
        print("p_value:", p_value)
    
        
        if p_value < 0.05:
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は小さい")
        else:
            # print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")
            pass

# print('')


# print('')

# print('')

# print('')

# print('')

# print('')

# print('等分散を仮定しない（welchのｔ）')



# for year in range(2018,2023):
# # year=2022
#     start_point = dt.datetime(year,1,1,0,0)
#     end_point = dt.datetime(year,12,31,23,30)

#     # まず、データがある分すべての相関をとる。
#     target =re_col[2:]
#     for name in target:
#         df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
#         df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


#         df['judge'] =0

#         df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
#         judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

#         df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

#         value0 = df[df['judge']==0][name].values
#         value1 = df[df['judge']==1][name].values
#         times =df['DateTime'].dt.date.values

#         X=value0
#         Y=value1
#         print('')
#         print(year,name)       
#         t_value, p_value = stats.ttest_ind(X, Y, alternative='greater',equal_var=False)
#         print("t_value:", t_value)
#         print("p_value:", p_value)
    
        
#         if p_value < 0.05:
#             print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は小さい")
#         else:
#             # print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")
#             pass







等分散を仮定

2018 出力低下量合計 (kW)
t_value: 18.437687589920372
p_value: 2.51142197020956e-75
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は小さい

2018 停止・低下ユニット数合計 (基)
t_value: 17.308419254339952
p_value: 9.929915697839251e-67
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は小さい

2018 時間前市場取引量 (kWh)
t_value: -29.139075810306448
p_value: 1.0

2018 スポット市場取引量 (kWh)
t_value: -10.53907229197732
p_value: 1.0

2018 太陽光誤差(実績-想定)合算 (kWh)
t_value: -0.1325662380131005
p_value: 0.5527308252030363

2018 風力誤差(実績-想定)合算値 (kWh)
t_value: -4.877362065542579
p_value: 0.9999994564926858

2019 出力低下量合計 (kW)
t_value: -2.2291705612575683
p_value: 0.9870924052924953

2019 停止・低下ユニット数合計 (基)
t_value: 1.0337057405848815
p_value: 0.15064400922236967

2019 時間前市場取引量 (kWh)
t_value: -20.15051351922978
p_value: 1.0

2019 スポット市場取引量 (kWh)
t_value: -1.4629910891368731
p_value: 0.9282561210251928

2019 太陽光誤差(実績-想定)合算 (kWh)
t_value: -2.075618760311446
p_value: 0.9810281103311098

2019 風力誤差(実績-想定)合算値 (kWh)
t_value: 0.8115179015753895
p_value: 0.2085396716587

C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_22392\3756409934.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2


t_value: -2.546507973712025
p_value: 0.9945554348020464

2020 風力誤差(実績-想定)合算値 (kWh)
t_value: -4.555230335263825
p_value: 0.9999973659926551

2021 出力低下量合計 (kW)
t_value: -18.88688844422721
p_value: 1.0

2021 停止・低下ユニット数合計 (基)
t_value: -16.062451709485583
p_value: 1.0

2021 時間前市場取引量 (kWh)
t_value: -27.620187844742514
p_value: 1.0

2021 スポット市場取引量 (kWh)
t_value: 0.03263628057579847
p_value: 0.4869825045898943

2021 太陽光誤差(実績-想定)合算 (kWh)
t_value: 4.072467151026489
p_value: 2.3361096830568183e-05
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は小さい

2021 風力誤差(実績-想定)合算値 (kWh)
t_value: -9.511089582848559
p_value: 1.0

2022 出力低下量合計 (kW)
t_value: 10.901851352601676
p_value: 6.924948195960775e-28
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は小さい

2022 停止・低下ユニット数合計 (基)
t_value: 9.453305523554961
p_value: 1.8435869925186385e-21
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は小さい

2022 時間前市場取引量 (kWh)
t_value: -23.03428125083708
p_value: 1.0

2022 スポット市場取引量 (kWh)
t_value: -20.7573973298456
p_value: 1.0

2022 太陽光誤差(実績-想定)合算 (kWh)
t_valu

In [4]:
# 広域予備率の検定

# 広域予備率


#１年毎正式版
cols = ['DateTime', 'gap_price[intra-spot](円/kWh)', '広域予備率平均値(%)']

df_select = df_king[cols]

re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)' , 'エリア毎の広域予備率の平均値(%)'
       ]
df_select.columns = re_col
# df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

percent =50
# これは各年

year= 2022

start_point = dt.datetime(year,1,1,0,0)
end_point = dt.datetime(year,12,31,23,30)

name ='エリア毎の広域予備率の平均値(%)'

df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


df['judge'] =0

df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

value0 = df[df['judge']==0][name].values
value1 = df[df['judge']==1][name].values



X=value0
Y=value1
print('')
print(year,name)       
t_value, p_value = stats.ttest_ind(X, Y, alternative='less',equal_var=True)
print("t_value:", t_value)
print("p_value:", p_value)


if p_value < 0.05:
       print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい")
else:
       print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")

t_value, p_value = stats.ttest_ind(X, Y, alternative='less',equal_var=False)
print("t_value:", t_value)
print("p_value:", p_value)


if p_value < 0.05:
       print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい")
else:
       print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")










2022 エリア毎の広域予備率の平均値(%)
t_value: 0.25571898637828977
p_value: 0.600914167344614
0.601 のため、帰無仮説が採択されました。あるとないに差はありません
t_value: 0.2557244128880196
p_value: 0.6009162615541468
0.601 のため、帰無仮説が採択されました。あるとないに差はありません


In [ ]:

#片側t検定　ある方がない方より大きい 
#１年毎正式版
cols = ['DateTime', 'gap_price[intra-spot](円/kWh)','低下量合計', '停止・低下ユニット数合計','intra_volume（MWh/h）','spot_volume(kWh)', '太陽光絶対値誤差合算(実績-想定)(kWh)',
       '風力絶対値誤差合算(実績-想定)(kWh)']

df_select = df_king[cols]

re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)','出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
       '風力誤差(実績-想定)合算値 (kWh)'
    #    , 'エリア毎の広域予備率の平均値(%)'
       ]
df_select.columns = re_col
df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

percent =50
# これは各年

# for year in range(2018,2023):

for year in range(2018,2023):
# year=2022
    start_point = dt.datetime(year,1,1,0,0)
    end_point = dt.datetime(year,12,31,23,30)

    # まず、データがある分すべての相関をとる。
    target =re_col[2:]
    for name in target:
        df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
        df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


        df['judge'] =0

        df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
        judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

        df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

        value0 = df[df['judge']==0][name].values
        value1 = df[df['judge']==1][name].values
        times =df['DateTime'].dt.date.values

        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, alternative='less')
        print("t_value:", t_value)
        print("p_value:", p_value)
    
        
        if p_value < 0.05:
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい")
        else:
            print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")







In [1]:
target

NameError: name 'target' is not defined

In [18]:
X=value0
Y=value1
nX =len(X)
nY = len(Y)

X_mean = X.mean()
Y_mean=Y.mean()

In [19]:
print(X_mean,Y_mean)

17619710.36133592 18427333.965931177


In [20]:
name

'スポット市場取引量 (kWh)'

In [24]:

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.weightstats import ttest_ind


In [26]:
X=value0
Y=value1
m =len(X)
n= len(Y)

# 標本平均
mean_X = X.mean()
mean_Y = Y.mean()
# 標本分散
var_X = X.var()
var_Y = Y.var()
# 不偏分散
dvar_X = X.var(ddof=1)
dvar_Y = Y.var(ddof=1)
# 標本標準偏差
std_X = X.std()
std_Y = X.std()
# 自由度
dof = m+n - 2
 
print("mean_X:", mean_X)
print("mean_Y:", mean_Y)
print("var_X: ", var_X)
print("var_Y: ", var_Y)
print("dvar_X: ", dvar_X)
print("dvar_Y: ", dvar_Y)
print("std_X: ", std_X)  # np.sqrt(var_X)
print("std_Y: ", std_Y)  # np.sqrt(var_Y)
print("dof: ", dof)

mean_X: 17619710.36133592
mean_Y: 18427333.965931177
var_X:  6491818071490.157
var_Y:  6768050114000.655
dvar_X:  6492558132829.816
dvar_Y:  6768823959200.061
std_X:  2547904.643327563
std_Y:  2547904.643327563
dof:  17518


In [27]:
# 推定した分散
v = ((m - 1) * dvar_X + (n - 1) * dvar_Y) / (m + n - 2)
print("v:", v)

# 推定した標準偏差
s = np.sqrt(v)
print("s:", s)

# 標準誤差
se = np.sqrt(v/m + v/n)
print("se:", se)

# t値（2標本t統計量）の算出
t_value = (mean_X - mean_Y) / se
# t_value = (mean_X - mean_Y) / (s * np.sqrt(1/m + 1/n))  # 上式と同じ
print("t_value:", t_value)

# p値の算出
p_value = min(stats.t.cdf(t_value, dof), 1-stats.t.cdf(t_value, dof)) * 2
print("p_value:", p_value)


# 推定した分散
v = ((m - 1) * dvar_X + (n - 1) * dvar_Y) / (m + n - 2)
print("v:", v)
 
# 推定した標準偏差
s = np.sqrt(v)
print("s:", s)
 
# 標準誤差
se = np.sqrt(v/m + v/n)
print("se:", se)
 
# t値（2標本t統計量）の算出
t_value = (mean_X - mean_Y) / se
# t_value = (mean_X - mean_Y) / (s * np.sqrt(1/m + 1/n))  # 上式と同じ
print("t_value:", t_value)
 
# p値の算出
p_value = min(stats.t.cdf(t_value, dof), 1-stats.t.cdf(t_value, dof)) * 2
print("p_value:", p_value)

v: 6630486030845.238
s: 2574973.015556714
se: 38907.74897072638
t_value: -20.757397329845602
p_value: 1.4421004873103496e-94
v: 6630486030845.238
s: 2574973.015556714
se: 38907.74897072638
t_value: -20.757397329845602
p_value: 1.4421004873103496e-94


In [28]:


t_value, p_value = stats.ttest_ind(X, Y, equal_var=True)
print("t_value:", t_value)
print("p_value:", p_value)
 
if p_value < 0.025:
    print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。AとBに差があります")
else:
    print(f"{p_value:.3f} のため、帰無仮説が採択されました。AとBに差はありません")

t_value: -20.7573973298456
p_value: 1.4421004873105138e-94
p = 0.000 のため、帰無仮説が棄却されました。AとBに差があります
